In [33]:
import os
log_dir = '/vol/vssp/mammo2/will/logs/new'
fCount = 0
while os.path.exists(os.path.join(log_dir, 'training', str(fCount))):
    fCount+=1
    
print(os.path.exists(os.path.join(log_dir, 'training')))
print(log_dir + 'hello')

True
/vol/vssp/mammo2/will/logs/newhello


In [1]:
# fold
import cv2
import glob
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
# For some reason I have to tell it to use TensorFlows dimension ordering
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.callbacks import TensorBoard
from time import time
import os
import tensorflow as tf
from keras import applications

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='/vol/vssp/mammo2/will/logs/new', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        fCount=0
        while os.path.exists(os.path.join(log_dir, 'training' + '_' + str(fCount))):
            fCount+=1
        training_log_dir = os.path.join(log_dir, 'training' + '_' + str(fCount))
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation' + '_' + str(fCount))

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()


def get_images(path, dataSpecs):
    fileList = glob.glob(path) #'BengaliBMPConvert/*.bmp'   
    num = len(fileList)
    dataSpecs['classLength'].append(len(fileList))
    x = np.array([(cv2.imread(fname)) for fname in fileList])
    return x

def get_labels_one_hot(num_classes, class_id, num_samples):
    x = np.zeros((num_samples, num_classes))
    x[np.arange(num_samples),class_id] = 1
    return x

def fourCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(385, 385, 1), data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    #model.add(Conv2D(32, (3,3), activation='relu'))   
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    return model
    
def fiveCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(385, 385, 1), data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  
    return model
    
    
def main():
    import keras
    print('keras version: ', keras.__version__)
    print('TensorFlow version: ', tf.__version__)
    print('\nLoad images...')
    
    # Get images and labels
    data = {'img': 0, 'label': 0}
    dataSpecs = {'classLength': []}
    dataSpecs['classLength'] = []
    data['img'] = np.concatenate((
            #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/0/*', dataSpecs), # Class 0
            #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/1/*', dataSpecs) # Class 1
            get_images('/user/HS204/wm0015/student/allCalcs/0/*', dataSpecs), # Class 0
            get_images('/user/HS204/wm0015/student/allCalcs/1/*', dataSpecs) # Class 1
    ))      

    # Normalise
    data['img'] = data['img']/255    
    # Print image    
    img_calc = data['img']   
    plt.imshow(img_calc[0], cmap='gray')
    plt.show()
    
    # Create one hot labels
    labels_bg = get_labels_one_hot(2, 0, dataSpecs['classLength'][0])  
    labels_calc = get_labels_one_hot(2, 1, dataSpecs['classLength'][1])
    data['label'] = np.concatenate((
            get_labels_one_hot(2, 0, dataSpecs['classLength'][0]), # Class 0 
            get_labels_one_hot(2, 1, dataSpecs['classLength'][1]) # Class 1
    ))
    # Drop from 3 colour channels to 1 (greyscale)
    if 1==1:
        data['img'] = data['img'][:,:,:,0]
        data['img'] = np.reshape(data['img'], (data['img'].shape[0],data['img'].shape[1],data['img'].shape[2],1))
        print('new data shape = ', data['img'].shape)
    
    valStats = []
    folds = 10
    for crossVal in range(folds):

        # Shuffle data
        seed = 33
    #     np.random.seed(seed) # Has to be set before each use of random
        shuffleMask = np.random.permutation(data['img'].shape[0])    
        data['img'] = data['img'][shuffleMask, :, :, :]
        data['label'] = data['label'][shuffleMask, :]

        # Split traing and validation data        
        splitRatio = 0.9
        splitPoint = math.floor(data['img'].shape[0]*splitRatio)
        train_data = {'img': data['img'][0:splitPoint], 'label': data['label'][0:splitPoint]}
        val_data = {'img': data['img'][splitPoint:], 'label': data['label'][splitPoint:]}
#         print('train_data[img].shape = ', train_data['img'].shape)
#         print('train_data[label].shape = ', train_data['label'].shape)
#         print('val_data[img].shape = ', val_data['img'].shape)
#         print('valdiation_data[label].shape = ', val_data['label'].shape)
#         print('dataSpecs = ', dataSpecs['classLength'])


        sgd = optimizers.SGD(lr=5, decay=1e-6, momentum=0.9, nesterov=True)
        adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #0.001
        
        model = fiveCNN()
        model.compile(loss='binary_crossentropy',
                    optimizer=adam,
                     metrics=['accuracy'])
        tensorboard = TensorBoard(log_dir='/vol/vssp/mammo2/will/logs/new'.format(time()), write_images=True)
        if folds == 1:
            head.summary()  

        # Train
        model.fit(train_data['img'], train_data['label'], 
                batch_size=100, epochs=150, verbose=1,
                callbacks=[TrainValTensorBoard(write_graph=False)],
                validation_data=(val_data['img'], val_data['label']))

        # Evaluate
        score = model.evaluate(val_data['img'], val_data['label'], verbose=0)
        valStats.append(score)
        print('The score is......\n', score)


    valStats = np.asarray(valStats)
    print('Validations: \n', valStats[:, 1])
    print('Average loss: ', sum(valStats[:, 0])/folds)
    print('Average validation: ', sum(valStats[:, 1])/folds)
    
if __name__ == "__main__":
    main()

Using TensorFlow backend.


keras version:  2.2.0
TensorFlow version:  1.7.1

Load images...


<Figure size 640x480 with 1 Axes>

new data shape =  (1620, 385, 385, 1)
Train on 1458 samples, validate on 162 samples
Epoch 1/150
1458/1458 [==============================] - 8s 5ms/step - loss: 0.6813 - acc: 0.5583 - val_loss: 0.6675 - val_acc: 0.5370
Epoch 2/150
1458/1458 [==============================] - 5s 3ms/step - loss: 0.6585 - acc: 0.6193 - val_loss: 0.5918 - val_acc: 0.7284
Epoch 3/150
1458/1458 [==============================] - 5s 3ms/step - loss: 0.6007 - acc: 0.6907 - val_loss: 0.5779 - val_acc: 0.8704
Epoch 4/150
1458/1458 [==============================] - 5s 3ms/step - loss: 0.5031 - acc: 0.7860 - val_loss: 0.6466 - val_acc: 0.7099
Epoch 5/150
1458/1458 [==============================] - 5s 3ms/step - loss: 0.3988 - acc: 0.8244 - val_loss: 0.3472 - val_acc: 0.8827
Epoch 6/150
1458/1458 [==============================] - 5s 4ms/step - loss: 0.4438 - acc: 0.8045 - val_loss: 0.5505 - val_acc: 0.7407
Epoch 7/150
1458/1458 [==============================] - 5s 4ms/step - loss: 0.3550 - acc: 0.8615 - val_l

1458/1458 [==============================] - 6s 4ms/step - loss: 9.1108e-05 - acc: 1.0000 - val_loss: 0.0556 - val_acc: 0.9753
Epoch 61/150
1458/1458 [==============================] - 6s 4ms/step - loss: 1.2429e-04 - acc: 1.0000 - val_loss: 0.0434 - val_acc: 0.9815
Epoch 62/150
1458/1458 [==============================] - 6s 4ms/step - loss: 1.0994e-04 - acc: 1.0000 - val_loss: 0.0333 - val_acc: 0.9877
Epoch 63/150
1458/1458 [==============================] - 6s 4ms/step - loss: 6.9591e-05 - acc: 1.0000 - val_loss: 0.0421 - val_acc: 0.9815
Epoch 64/150
1458/1458 [==============================] - 6s 4ms/step - loss: 9.7035e-05 - acc: 1.0000 - val_loss: 0.0612 - val_acc: 0.9753
Epoch 65/150
1458/1458 [==============================] - 6s 4ms/step - loss: 5.8505e-05 - acc: 1.0000 - val_loss: 0.0422 - val_acc: 0.9815
Epoch 66/150
1458/1458 [==============================] - 6s 4ms/step - loss: 8.4135e-05 - acc: 1.0000 - val_loss: 0.0497 - val_acc: 0.9753
Epoch 67/150
1458/1458 [=========

...

1458/1458 [==============================] - 5s 4ms/step - loss: 0.0075 - acc: 0.9986 - val_loss: 0.1006 - val_acc: 0.9630
Epoch 26/150
1458/1458 [==============================] - 5s 3ms/step - loss: 0.0047 - acc: 0.9979 - val_loss: 0.0444 - val_acc: 0.9877
Epoch 27/150
1458/1458 [==============================] - 5s 3ms/step - loss: 0.0045 - acc: 0.9993 - val_loss: 0.0232 - val_acc: 0.9938
Epoch 28/150
1458/1458 [==============================] - 5s 4ms/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0383 - val_acc: 0.9938
Epoch 29/150
1458/1458 [==============================] - 5s 3ms/step - loss: 0.0022 - acc: 0.9993 - val_loss: 0.0640 - val_acc: 0.9938
Epoch 30/150
1458/1458 [==============================] - 5s 4ms/step - loss: 0.0025 - acc: 0.9993 - val_loss: 0.0124 - val_acc: 0.9938
Epoch 31/150
1458/1458 [==============================] - 5s 3ms/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0528 - val_acc: 0.9877
Epoch 32/150
1458/1458 [==============================] - 5s 

1458/1458 [==============================] - 5s 3ms/step - loss: 5.1756e-05 - acc: 1.0000 - val_loss: 0.0423 - val_acc: 0.9938
Epoch 85/150
1458/1458 [==============================] - 5s 4ms/step - loss: 9.5528e-05 - acc: 1.0000 - val_loss: 0.0423 - val_acc: 0.9938
Epoch 86/150
1458/1458 [==============================] - 5s 4ms/step - loss: 4.0979e-05 - acc: 1.0000 - val_loss: 0.0447 - val_acc: 0.9938
Epoch 87/150
1458/1458 [==============================] - 5s 4ms/step - loss: 4.3575e-05 - acc: 1.0000 - val_loss: 0.0434 - val_acc: 0.9938
Epoch 88/150
1458/1458 [==============================] - 5s 4ms/step - loss: 7.1784e-05 - acc: 1.0000 - val_loss: 0.0445 - val_acc: 0.9938
Epoch 89/150
1458/1458 [==============================] - 5s 4ms/step - loss: 4.6166e-05 - acc: 1.0000 - val_loss: 0.0517 - val_acc: 0.9877
Epoch 90/150
1458/1458 [==============================] - 5s 3ms/step - loss: 5.1800e-05 - acc: 1.0000 - val_loss: 0.0500 - val_acc: 0.9877
Epoch 91/150
1458/1458 [=========

1458/1458 [==============================] - 5s 3ms/step - loss: 3.3254e-05 - acc: 1.0000 - val_loss: 0.0178 - val_acc: 0.9938
Epoch 143/150
1458/1458 [==============================] - 5s 4ms/step - loss: 1.7794e-04 - acc: 1.0000 - val_loss: 0.0559 - val_acc: 0.9877
Epoch 144/150
1458/1458 [==============================] - 5s 4ms/step - loss: 1.2491e-04 - acc: 1.0000 - val_loss: 0.0945 - val_acc: 0.9753
Epoch 145/150
1458/1458 [==============================] - 5s 4ms/step - loss: 8.7663e-06 - acc: 1.0000 - val_loss: 0.0607 - val_acc: 0.9877
Epoch 146/150
1458/1458 [==============================] - 5s 3ms/step - loss: 8.5762e-06 - acc: 1.0000 - val_loss: 0.0553 - val_acc: 0.9938
Epoch 147/150
1458/1458 [==============================] - 5s 3ms/step - loss: 8.2086e-06 - acc: 1.0000 - val_loss: 0.0541 - val_acc: 0.9938
Epoch 148/150
1458/1458 [==============================] - 5s 4ms/step - loss: 1.0839e-05 - acc: 1.0000 - val_loss: 0.0536 - val_acc: 0.9938
Epoch 149/150
1458/1458 [==

1458/1458 [==============================] - 5s 4ms/step - loss: 3.8748e-05 - acc: 1.0000 - val_loss: 0.0312 - val_acc: 0.9877
Epoch 51/150
1458/1458 [==============================] - 6s 4ms/step - loss: 9.1053e-05 - acc: 1.0000 - val_loss: 0.0331 - val_acc: 0.9877
Epoch 52/150
1458/1458 [==============================] - 5s 4ms/step - loss: 5.7827e-05 - acc: 1.0000 - val_loss: 0.0332 - val_acc: 0.9877
Epoch 53/150
1458/1458 [==============================] - 5s 4ms/step - loss: 3.6152e-05 - acc: 1.0000 - val_loss: 0.0319 - val_acc: 0.9877
Epoch 54/150
1458/1458 [==============================] - 5s 4ms/step - loss: 8.3932e-05 - acc: 1.0000 - val_loss: 0.0336 - val_acc: 0.9938
Epoch 55/150
1458/1458 [==============================] - 5s 4ms/step - loss: 2.7345e-05 - acc: 1.0000 - val_loss: 0.0317 - val_acc: 0.9877
Epoch 56/150
1458/1458 [==============================] - 5s 3ms/step - loss: 6.3763e-05 - acc: 1.0000 - val_loss: 0.0322 - val_acc: 0.9877
Epoch 57/150
1458/1458 [=========

1458/1458 [==============================] - 5s 4ms/step - loss: 9.0910e-06 - acc: 1.0000 - val_loss: 0.0312 - val_acc: 0.9938
Epoch 109/150
1458/1458 [==============================] - 5s 3ms/step - loss: 1.6085e-05 - acc: 1.0000 - val_loss: 0.0335 - val_acc: 0.9938
Epoch 110/150
1458/1458 [==============================] - 5s 3ms/step - loss: 1.0093e-05 - acc: 1.0000 - val_loss: 0.0328 - val_acc: 0.9938
Epoch 111/150
1458/1458 [==============================] - 5s 3ms/step - loss: 1.4075e-05 - acc: 1.0000 - val_loss: 0.0333 - val_acc: 0.9938
Epoch 112/150
1458/1458 [==============================] - 5s 3ms/step - loss: 8.1104e-06 - acc: 1.0000 - val_loss: 0.0310 - val_acc: 0.9938
Epoch 113/150
1458/1458 [==============================] - 5s 4ms/step - loss: 1.1389e-05 - acc: 1.0000 - val_loss: 0.0308 - val_acc: 0.9938
Epoch 114/150
1458/1458 [==============================] - 5s 4ms/step - loss: 1.0919e-05 - acc: 1.0000 - val_loss: 0.0321 - val_acc: 0.9938
Epoch 115/150
1458/1458 [==

In [2]:
print('hello world')

hello world


In [12]:
x = [[0.6940992025681484, 0.3950617283950617], [0.6884314866713536, 0.5185185185185185], [0.6855149121932042, 0.5679012345679012]]
print(x[0])
print(sum(x[:][1])/2)

[0.6940992025681484, 0.3950617283950617]
0.603475002594936


In [35]:
x = [[6.5308382717179665, 0.5925925925925926], [0.16470135086112553, 0.9444444444444444], [0.15422181794304907, 0.9444444444444444], [0.19875195217721256, 0.9382716049382716]] 
x = np.asarray(x)
print(x, '\n')
print(x[0][0])
print(x[:, 0])
print(x.shape)

[[6.53083827 0.59259259]
 [0.16470135 0.94444444]
 [0.15422182 0.94444444]
 [0.19875195 0.9382716 ]] 

6.5308382717179665
[6.53083827 0.16470135 0.15422182 0.19875195]
(4, 2)
